In [5]:
from utils_mar import *
from utilsNew import *
from utils import *

In [7]:
from IPython import display
import random
import numpy as np
import torch
import pickle
import timeit
import time
import argparse
import pprint
from pathlib import Path
from confs import fn, fn2, fn22, LogFn
#from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm
import numpy.random as npr

In [8]:
import matplotlib.pyplot as plt
import pickle

In [10]:
cudaid = 2
loglv = 2
torch.cuda.set_device(cudaid)
#------------------------------------------------------------------------------------
# fix the random seed for several packages
torch.manual_seed(0) # cpu
torch.cuda.manual_seed(0) #gpu
np.random.seed(0) #numpy
random.seed(0) #random and transforms
torch.backends.cudnn.deterministic=True # cudnn

In [11]:
cuda = torch.cuda.is_available()
if cuda:
    torch.set_default_tensor_type(torch.cuda.DoubleTensor)

In [12]:
conDenfs = [fn, fn2, fn22]
f = fn
f2 = fn2

In [13]:
n = m = 400
p = 50
prob = 0.2

sigmaY = 1.0

# generate the parameters
beta0 = torch.cat((torch.tensor([1.0, 0, 2, 0, -3, -4, 5]), torch.zeros(p-7))) 
bTheta0 = genbTheta(n, m, sigVs=np.array([10, 9, 8, 7, 6])*10) 
initbetapref = 1 + (torch.rand(p)-1/2)/40  #[0.875, 1.125]
initthetapref = 1 + (torch.rand(n, m)-1/2)/4
betainit = beta0 * initbetapref
bThetainit = bTheta0 * initthetapref
#betainit = torch.cat((torch.tensor([0.8, 0, 1.5, 0, -2.5, -4.3, 4.8]), torch.zeros(p-7))) * 10
#bThetainit = genbTheta(n, m, sigVs=np.array([15, 10, 7, 5, 5])/2) 

#X = genXUnif(n, m, p) 
X = genXBin(n, m, p, prob=prob) 
Y = genYnorm(X, bTheta0, beta0, sigmaY)
# around 95% missing rate
inps = {}
inps[50] = - 8.0
inps[100] =  -5.93
inps[200] = -5.30
inps[400] = -5.2
inps[800] = -5.15
inps[1600] = -5.1
R = genR(Y, "linear", inp=inps[n], slop=1)
R.to_dense().mean()

tensor(0.0502)

In [15]:
v1 = lossLpb(bTheta0, beta0, conDenfs, X, Y, R, fct=10, N=10000)

In [16]:
v1 = marLossLpT(bTheta0, beta0, conDenfs, X, Y, R)
v2 = lossLpT(bTheta0, beta0, conDenfs, X, Y, R)

In [17]:
torch.norm(v1)

tensor(2.6509e-05)

In [15]:
torch.norm(v1  - v2)

tensor(0.0004)

In [106]:
etaT = 0.1
CT = 20

In [107]:
bThetaOld = bTheta0
betaNew = beta0
LamT = LamTfn(CT, n, m, p)

In [108]:
LpTvOld = lossLpT(bThetaOld, betaNew, conDenfs, X, Y, R)
svdres = torch.svd(bThetaOld-LpTvOld*etaT)
U, S, V =  svdres.U, svdres.S, svdres.V
softS = (S-LamT*etaT).clamp_min(0)
bThetaNew1 = U.matmul(torch.diag(softS)).matmul(V.t())

In [109]:
bThetaOld = bTheta0
betaNew = beta0
LamT = LamTfn(CT, n, m, p)

In [110]:
LpTvOld = marLossLpT(bThetaOld, betaNew, conDenfs, X, Y, R)
svdres = torch.svd(bThetaOld-LpTvOld*etaT)
U, S, V =  svdres.U, svdres.S, svdres.V
softS = (S-LamT*etaT).clamp_min(0)
bThetaNew2 = U.matmul(torch.diag(softS)).matmul(V.t())

In [111]:
torch.norm(bThetaNew1 - bThetaNew2)

tensor(0.0004)

In [43]:
# calaulte the updated beta under MAR for linear
# I didn't check it before. 
# the beta is before soft-thresholding
def marLinearUpdateBeta(X, Y, R, bTheta):
    if not R.is_sparse:
        R = R.to_sparse()
    if not X.is_sparse:
        X = X.to_sparse()
    YvecP = Y[R.to_dense().bool()] # M 
    bThetaVecP = bTheta[R.to_dense().bool()] # M 
    XvecP = X.to_dense()[R.to_dense().bool()].to_sparse() #M x p
    itm1 = ((YvecP-bThetaVecP).reshape(-1, 1) * XvecP.to_dense()).sum(axis=0)
    itm2inv = XvecP.to_dense().T.matmul(XvecP.to_dense())
    itm2 = torch.inverse(itm2inv)
    betaHat = itm1.matmul(itm2)
    return betaHat

In [20]:
beta = beta0
bThetaOld = bTheta0

In [36]:
# calaulte the updated bTheta under MNAR for linear, approximately
# I didn't check it before. 
# the theta is before soft-thresholding
def mnarLinearUpdateThetaApprox(X, Y, R, bThetaOld, beta, sigma=5):
    bThetaNew = bThetaOld.clone()
    betaX = X.to_dense().matmul(beta)
    mu = torch.mean(betaX)
    sigma2sq = torch.var(betaX)
    sigma1sq = sigma**2
    YvecP = Y[R.to_dense().bool()] # M 
    betaXvecP = betaX[R.to_dense().bool()]
    bThetaNewVecP = (YvecP * sigma2sq + mu * sigma1sq  - (sigma1sq+sigma2sq)*betaXvecP)/sigma2sq
    bThetaNew[R.to_dense().bool()] = bThetaNewVecP
    return bThetaNew

In [38]:
# calaulte the updated bTheta under MAR for linear
# I didn't check it before. 
# the theta is before soft-thresholding
def marLinearUpdateTheta(X, Y, R, bThetaOld, beta):
    bThetaNew = bThetaOld.clone()
    betaX = X.to_dense().matmul(beta)
    YvecP = Y[R.to_dense().bool()] # M 
    betaXvecP = betaX[R.to_dense().bool()]
    bThetaNewVecP = YvecP  - betaXvecP
    bThetaNew[R.to_dense().bool()] = bThetaNewVecP
    return bThetaNew

In [40]:
sigma = 5

In [42]:
beta = beta0
bTheta = bTheta0

In [49]:
# calculate the loss under MAR 
# unchecked
def marLinearLoss(X, Y, R, bTheta, beta, sigma):
    n, m = Y.shape
    YvecP = Y[R.to_dense().bool()] # M 
    bThetaVecP = bTheta[R.to_dense().bool()] # M 
    betaX = X.to_dense().matmul(beta)
    betaXvecP = betaX[R.to_dense().bool()]
    res = torch.sum(-(YvecP - bThetaVecP - betaXvecP)**2/2/sigma**2)/n/m
    return res

In [48]:
marLinearLoss(X, Y, R, bTheta, beta, sigma)

tensor(-0.0014)